All of this was done with a nice little explaining of the 1D example of: https://www.youtube.com/watch?v=S-6Z8N-30AU

Now in 2D:

Deriving the algorithm with the Maxwell equations:

\begin{align}
\nabla \times \mathbf{E} &= -\,\mu\,\frac{\partial \mathbf{H}}{\partial t} \quad &&\text{(Faraday's law)} \\[6pt]
\nabla \times \mathbf{H} &= \mathbf{J} + \varepsilon\,\frac{\partial \mathbf{E}}{\partial t} \quad &&\text{(Ampère–Maxwell law)} \\[6pt]
\nabla \cdot \mathbf{E} &= \frac{\rho}{\varepsilon} \quad &&\text{(Gauss's law for electricity)} \\[6pt]
\nabla \cdot \mathbf{H} &= 0 \quad &&\text{(Gauss's law for magnetism)}
\end{align}

After looking at the Maxwell equations we see that the equations (3) and (4) are not necessary for our numerical simulation, hence we will be left with: 

\begin{align}
\nabla \times \mathbf{E} &= -\,\mu\,\frac{\partial \mathbf{H}}{\partial t} \quad &&\text{(Faraday's law)} \\[6pt]
\nabla \times \mathbf{H} &= \mathbf{J} + \varepsilon\,\frac{\partial \mathbf{E}}{\partial t} \quad &&\text{(Ampère–Maxwell law)} \\[6pt]
\end{align}

We now know that the an approximation of the derivative is:
\begin{align}
\frac{df}{dx}(x_0) 
&= \lim_{h \to 0} \frac{f\!\left(x_0 + \tfrac{h}{2}\right) - f\!\left(x_0 - \tfrac{h}{2}\right)}{h} \\[6pt]
&= \lim_{\Delta x \to 0} \frac{f\!\left(x_0 + \tfrac{\Delta x}{2}\right) - f\!\left(x_0 - \tfrac{\Delta x}{2}\right)}{\Delta x} \\[6pt]
&\approx \frac{f\!\left(x_0 + \tfrac{\Delta x}{2}\right) - f\!\left(x_0 - \tfrac{\Delta x}{2}\right)}{\Delta x},
\quad \Delta x \ll 1
\end{align}

Now we can also shift this derivative by $+\tfrac{\Delta x}{2}$ and get: 

\begin{align}
\frac{df}{dx}(x_0+\tfrac{\Delta x}{2})
&\approx \frac{f\!\left(x_0 + \Delta x\right) - f\!\left(x_0\right)}{\Delta x},
\quad \Delta x \ll 1
\end{align}

Now we want to use this derivatives to simplify our maxwell equations. For this we first need to think of a grid in space and time we want to use. One way as demonstrated in the video is to changingly look at H and E with a difference of $\tfrac{\Delta x}{2}$ and so we will be looking at H in every half step in time and space and we will look at E on every whole step in time and space. This means that we will start with the initial condition of $E(0)$ and then look at $H(\tfrac{\Delta x}{2})$ then look at $E(\Delta x)$ and so on.

The last thing that we now need, is the maxwell equations in 2D as the curl is usually a 3D operator. We now only look at an x polarized electric field, which means $E_y = E_z = 0$ and we will only assume that the EM-wave only propagates in the y and z direction ($k_y$, $k_z$). Note that with this the magnetic field obviously has to be polarized in the z or y direction depending if $k_y$ or $k_z$ or a linear combination is used. With this the curl of the electric field is for example: $$\nabla \times E = (0,\tfrac{\partial E_x}{\partial z},-\tfrac{\partial E_x}{\partial y})$$ and the magnetic field becomes: $$\nabla \times H = (\tfrac{\partial H_z}{\partial y} - \tfrac{\partial H_y}{\partial z},0,0)$$

With all of these simplifications our new maxwell equations become:
\begin{align}
\boxed{\;\frac{\partial H_y}{\partial t}(y,z,t) =-\frac{1}{\mu}\,\frac{\partial E_x}{\partial z}(y,z,t)\;} \\[6pt]
\boxed{\;\frac{\partial H_z}{\partial t}(y,z,t) =\frac{1}{\mu}\,\frac{\partial E_x}{\partial y}(y,z,t)\;} \\[6pt]
\boxed{\;\frac{\partial E_x}{\partial t}(y,z,t) =\frac{1}{\varepsilon}\,\left(\frac{\partial H_z}{\partial y}(y,z,t)-\frac{\partial H_y}{\partial z}(y,z,t)\right)\;}
\end{align}

Lets now talk about the last thing missing, the parametrization of the derivatives and the Yee grid.
WWe know $H_z(y)$ is defined at half steps $(t + \tfrac{\Delta t}{2}, y + \tfrac{\Delta y}{2})$ and $E_x(y)$ is defined at whole steps $(t,y)$. Now we just say that $y$ and $t$ can be parametrized with $j\,\Delta y$ and $n\,\Delta t$ where n and j are independent integer steps. With this we can then say that e.g. $H_z(y + \tfrac{\Delta y}{2}) = H_z(\Delta y\,(j + \tfrac{1}{2}))$ and $E_x(y) = H_z(j\,\Delta y)$. To simplify this halfstep and step thing we will now just shift the j and n index with $-\tfrac{1}{2}$.

With inserting all of this and with a little math we get this:

\begin{align}
\boxed{\;H_z(i,j,n) = H_z(i,j,n-1)+\frac{\Delta t}{\mu \Delta y} \left(E_x(i,j+1,n)-E_x(i,j,n)\right);} \\[6pt]
\boxed{\;H_y(i,j,n) = H_y(i,j,n-1)-\frac{\Delta t}{\mu \Delta z} \left(E_x(i+1,j,n)-E_x(i,j,n)\right);} \\[6pt]
\boxed{\;E_x(i,j,n+1) = E_x(i,j,n) + \frac{\Delta t}{\epsilon \Delta y} \left(H_z(i,j,n)-H_z(i,j-1,n)\right) - \frac{\Delta t}{\epsilon \Delta z} \left(H_y(i,j,n)-H_y(i-1,j,n)\right)\;}
\end{align}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

#Constants

eps0 = 8.8541878128e-12
mu0 = 1.256637062e-6
c = 1/np.sqrt(eps0*mu0)
imp0 = np.sqrt(mu0/eps0)

# values for other than vakuum, epsilon (eps) also can be an array of permativities in the y direction for every step, mu also can be so

mu = mu0
eps = eps0
v = 1/np.sqrt(eps*mu)
imp = np.sqrt(mu/eps)

j_max = 500  #size of y
i_max = 500 #size of z
n_max = 2000  #size of t
j_source = 250 #space step of j_source
i_source = 250 #space step of i_source

E_x = np.zeros(shape=(i_max,j_max))
H_z = np.zeros(shape=(i_max,j_max))
H_y = np.zeros(shape=(i_max,j_max))

#can be replaced with just E_x and H_z and is not important but or simplicity, it can be better do define it

E_x_prev = np.zeros(shape=(i_max,j_max))
H_z_prev = np.zeros(shape=(i_max,j_max))
H_y_prev = np.zeros(shape=(i_max,j_max))

lambda_min = 350e-9  #minimum wavelength
dy = lambda_min/20
dz = lambda_min/20
S=0.5
dt = S/((np.sqrt(1/dy**2+1/dz**2))*c)

# Source function (for demonstration)
def Source_Func(t):
    tau = 30   # in time steps
    t_0 = tau*3     # delay for source to work
    lambda_0 = 550e-9  #defines the frequency of the source
    w0 = 2*np.pi*c/lambda_0 
    return np.exp(-(t-t_0)**2/tau**2)*np.sin(w0*t*dt)

# Set up the figure and axis
#fig, ax = plt.subplots()
#line, = ax.plot(E_x, lw=2)
#ax.set_ylim([-1, 1])
#ax.set_xlim([0, j_max])
#ax.set_xlabel("Grid index j")
#ax.set_ylabel("E-field amplitude")

# The update function for animation
for n in range(n_max):
    # Update magnetic field z
    #Boundary condition for propagating of screen
    #H_z[j_max-1] = H_z_prev[j_max-2]
    
    for i in range(i_max-1):
        for j in range(j_max-1):
            H_z[i,j] = H_z_prev[i,j] + dt/(dy*mu0) * (E_x[i,j+1] - E_x[i,j])
    H_z_prev = H_z

    # Add magnetic field source
    H_z[i_source-1,j_source-1] -= Source_Func(n)/imp0
    H_z_prev[i_source-1,j_source-1] = H_z[i_source-1,j_source-1]
    
    # Update magnetic field z
    #Boundary condition for propagating of screen
    #H_z[j_max-1] = H_z_prev[j_max-2]
    
    for j in range(j_max-1):
        for i in range(i_max-1):
            H_y[i,j] = H_y_prev[i,j] - dt/(dz*mu0) * (E_x[i+1,j] - E_x[i,j])
    H_y_prev = H_y

    # Add magnetic field source
    H_y[i_source-1,j_source-1] -= Source_Func(n)/imp0
    H_y_prev[i_source-1,j_source-1] = H_y[i_source-1,j_source-1]

    # Update electric field
    #Boundary condition for propagating of screen
    #E_x[0] = E_x_prev[1]
    for i in range(1,i_max):
        for j in range(1,j_max):
            E_x[i,j] = E_x_prev[i,j] + dt/(dy*eps) * (H_z[i,j] - H_z[i,j-1]) - dt/(dz*eps) * (H_y[i,j] - H_y[i-1,j])
    E_x_prev = E_x

    # Add electric field source
    E_x[i_source,j_source] += Source_Func(n+1)
    E_x_prev[i_source,j_source] = E_x[i_source,j_source]



# Create the animation
#ani = FuncAnimation(fig, update, frames=n_max, interval=20, blit=True)

    if n%10 == 0:
        plt.figure(figsize=(6,5))
        plt.imshow(E_x.T, origin='lower', cmap='RdBu', interpolation='bilinear')
        plt.colorbar(label=r'$E_x$ amplitude')
        plt.title(f'E-field after {n_max} steps')
        plt.xlabel('y-index')
        plt.ylabel('z-index')
        plt.tight_layout()
        plt.show()


This is working, but is very slow, and has no boundary conditions, so it basically is a mirror, lets vectorize it!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

#Constants

eps0 = 8.8541878128e-12
mu0 = 1.256637062e-6
c = 1/np.sqrt(eps0*mu0)
imp0 = np.sqrt(mu0/eps0)

# values for other than vakuum, epsilon (eps) also can be an array of permativities in the y direction for every step, mu also can be so

mu = mu0
eps = eps0
v = 1/np.sqrt(eps*mu)
imp = np.sqrt(mu/eps)

j_max = 500  #size of y
i_max = 500 #size of z
n_max = 2000  #size of t
j_source = 250 #space step of j_source
i_source = 250 #space step of i_source

E_x = np.zeros(shape=(i_max,j_max))
H_z = np.zeros(shape=(i_max,j_max))
H_y = np.zeros(shape=(i_max,j_max))

#can be replaced with just E_x and H_z and is not important but or simplicity, it can be better do define it

E_x_prev = np.zeros(shape=(i_max,j_max))
H_z_prev = np.zeros(shape=(i_max,j_max))
H_y_prev = np.zeros(shape=(i_max,j_max))

lambda_min = 350e-9  #minimum wavelength
dy = lambda_min/20
dz = lambda_min/20
S=0.5
dt = S/((np.sqrt(1/dy**2+1/dz**2))*c)

# Source function (for demonstration)
def Source_Func(t):
    tau = 30   # in time steps
    t_0 = tau*3     # delay for source to work
    lambda_0 = 550e-9  #defines the frequency of the source
    w0 = 2*np.pi*c/lambda_0 
    return np.exp(-(t-t_0)**2/tau**2)*np.sin(w0*t*dt)

# Set up the figure and axis
#fig, ax = plt.subplots()
#line, = ax.plot(E_x, lw=2)
#ax.set_ylim([-1, 1])
#ax.set_xlim([0, j_max])
#ax.set_xlabel("Grid index j")
#ax.set_ylabel("E-field amplitude")

# The update function for animation
for n in range(n_max):
    # Update magnetic field z
    #BOUNDARY CONDITIONS from 1D
    #H_z[i,j_max-1] = H_z_prev[i,j_max-2]
    #H_y[i_max-1,j] = H_y_prev[i_max-2,j]
    #E_x[0] = E_x_prev[1] #is 1D
    
    #MAXWELL EQUATIONS
    
    # Update magnetic field z
    H_z[:,:j_max-1] = H_z_prev[:,:j_max-1] + dt/(dy*mu0) * (E_x[:,1:j_max] - E_x[:,:j_max-1])
    H_z_prev = H_z
    
    # Update magnetic field y
    H_y[:i_max-1,:] = H_y_prev[:i_max-1,:] - dt/(dz*mu0) * (E_x[1:i_max,:] - E_x[:i_max-1,:])
    H_y_prev = H_y

    # Update electric field
    E_x[1:,1:] = E_x_prev[1:,1:] + dt/(dy*eps) * (H_z[1:,1:] - H_z[1:,:j_max-1]) - dt/(dz*eps) * (H_y[1:,1:] - H_y[:i_max-1,1:])
    E_x_prev = E_x

    #SOURCE
    # Add magnetic field source of H_z
    #H_z[i_source-1,j_source-1] -= Source_Func(n)/imp0
    #H_z_prev[i_source-1,j_source-1] = H_z[i_source-1,j_source-1]
    
    # Add magnetic field source H_y
    #H_y[i_source-1,j_source-1] -= Source_Func(n)/imp0
    #H_y_prev[i_source-1,j_source-1] = H_y[i_source-1,j_source-1]
    
    # Add electric field source E_x, If you want radial wave, only use E_x, for a directional wave, define H_y and H_z
    E_x[i_source,j_source] += Source_Func(n+1)
    E_x_prev[i_source,j_source] = E_x[i_source,j_source]



# Create the animation
#ani = FuncAnimation(fig, update, frames=n_max, interval=20, blit=True)

    if n%10 == 0:
        plt.figure(figsize=(6,5))
        plt.imshow(E_x.T, origin='lower', cmap='RdBu', interpolation='bilinear')
        plt.colorbar(label=r'$E_x$ amplitude')
        plt.title(f'E-field after {n_max} steps')
        plt.xlabel('y-index')
        plt.ylabel('z-index')
        plt.tight_layout()
        plt.show()


This is like 1mio times faster, but I am still unsure of which boundary condition I want to use.